# Local validation

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split


df = pd.read_json('./data/train_v2.jsonl', lines=True)
train, test = train_test_split(df, test_size=0.2)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

train.head()

,index,text,acronym,options
0,374,SAL signal automatique lumineux SGTC service g...,STD,"{'Système de Transmission de Données': False, ..."
1,282,1.3. Résumé des modifications Suppression des ...,AC,"{'Amélioration Continue': False, 'Autorité de ..."
2,180,Historique TGV310000 25kV / compatibilité : O...,US,"{'Unité de signalisation': False, 'Ultra Son':..."
3,453,Unité Documentation d’exploitation directions...,BHR,"{'BLESLE': False, 'Bureau horaires régional': ..."
4,396,"AUTOR rame automotrice à traction thermique, d...",EF,"{'Essai journalier (des freins)': False, 'Esca..."


In [2]:
answers = []
for _, row in test.iterrows():
    ans = []
    for index, value in enumerate(row['options'].values()):
        if(value):
            ans.append(index)
    answers.append(ans)
test["answers"] = answers
test.head()

,index,text,acronym,options,answers
0,441,Poste de commande à distance de Saint Sulpice ...,PCD,{'Protection de courte durée : méthode de prot...,[1]
1,25,EM engin moteur EP embranchement particulier E...,EM,"{'État membre': False, 'Étude mécanique': Fals...",[3]
2,321,COGC centre opérationnel de gestion des circul...,PN,"{'PONS': False, 'Paris Nord': False, 'Période ...",[6]
3,290,En application du document de principe RFN NG ...,NG,"{'Nouvelle Génération': False, 'Notice général...",[1]
4,294,Réservé. Article A106 Circulation et régulatio...,CCL,{'Commande Centralisée des Locomotives ': Fal...,[2]


In [3]:
import numpy as np
from sentence_transformers import SentenceTransformer
import torch

torch.cuda.empty_cache()

embedder = SentenceTransformer("intfloat/multilingual-e5-base", device="cpu")

train_embs = embedder.encode(
    train['text'].tolist(),
    batch_size=32,
    convert_to_numpy=True,
    normalize_embeddings=True,
    show_progress_bar=True
)

test_embs = embedder.encode(
    test['text'].tolist(),
    batch_size=32,
    convert_to_numpy=True,
    normalize_embeddings=True,
    show_progress_bar=True
)

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|██████████| 4/4 [00:02<00:00,  1.96it/s]


In [4]:
def top_k_examples(k):
    topk_indices = []
    for q_idx, row in test.iterrows():
        acronym = row['acronym']
        # Filter same acronym in train_df
        subdf = train[train['acronym'] == acronym]
        if len(subdf) == 0:
        # fallback: random examples
            topk_indices.append(train.sample(k).index.to_list())
            continue
    
        subset_indices = subdf.index.to_list()
        subset_embs = train_embs[subset_indices]
        
        # Cosine similarity (dot product of normalized embeddings)
        sims = np.dot(subset_embs, test_embs[q_idx].reshape(-1,1)).squeeze()
        topk_idx = np.argsort(-sims)[:k]
        topk_indices.append([subset_indices[i] for i in topk_idx])
    
    return topk_indices

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import os

torch.cuda.empty_cache()

model_name = "Qwen/Qwen2.5-32B-Instruct" #"mistralai/Mistral-Small-24B-Instruct-2501" #"mistralai/Mistral-7B-Instruct-v0.3"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto",
    offload_folder="./offload",
    torch_dtype=torch.float16,
)

pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    torch_dtype=torch.float16
)

pipe.tokenizer.pad_token_id = model.config.eos_token_id
pipe.model.config.use_cache = False

print("model loaded")

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 17/17 [15:29<00:00, 54.66s/it]
`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0


model loaded


In [6]:
def create_prompt_with_examples(acronym, text, options, examples, k):
    system = """Tu es un modèle expert en expansion d'acronymes ferroviaires.
Ton rôle est d'identifier la ou les définitions correctes d'un acronyme dans un texte.
Réponds uniquement avec une liste Python d'indices, ex. [0] ou [1, 2] ou []. \nExemples:"""

    
    for idx in examples: 
        example = train.iloc[idx]
        system += f'\nTexte exemple : "{example["text"]}\nAcronyme: {example["acronym"]}"\nOptions: '
        for j, opt in enumerate(example['options'].keys()):
            system += f'\n{j}. : {opt}'
        system += f'\nReponse correcte : {[i for i, value in enumerate(example["options"].values()) if value]}\n'
    user = f'Texte : "{text}"\nAcronyme : {acronym}\n'
    for i, opt in enumerate(options):
        user += f"Option {i} : {opt}\n"
    user += "Réponds avec la liste des numéros corrects :"
    prompt = [
    {"role": "system", "content": system},
    {"role": "user", "content": user}]
    chat = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
    return chat

In [7]:
import re

def extract_predicted_ids(outputs):
    predicted_ids = []
    for output in outputs:
        # Get the text after [/INST]
        text = output[0]["generated_text"].split("<|im_start|>assistant")[1] #"[/INST]"
        
        # Find all numeric patterns, including decimals
        numbers = re.findall(r'\d+(?:\.\d+)?', text)
        
        # Convert to ints safely, remove duplicates, and filter < 15
        ids = list(set(int(float(i)) for i in numbers if float(i) < 15))
        
        predicted_ids.append(ids)
    return predicted_ids

In [8]:
def predict(k):
    exampless = top_k_examples(k)
    sum = 0
    for examples in exampless:
        sum+= len(examples)
    print(f"average examples number: {sum/len(exampless)}")
    inputs = []
    for indexx, row in test.iterrows():
        prompt = create_prompt_with_examples(row['acronym'], row['text'], row['options'].keys(), exampless[indexx], k)
        inputs.append(prompt)
    outputs = pipe(inputs, do_sample=False, batch_size=4)
    predicted_ids = extract_predicted_ids(outputs)
    correct = 0
    for index, ids in enumerate(predicted_ids):
        if ids == test['answers'].iloc[index]:
            correct += 1
    accuracy = correct / len(predicted_ids)
    print(f"accuracy: {accuracy}")

### k?

In [9]:
predict(8)

average examples number: 7.0606060606060606


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


accuracy: 0.7474747474747475


In [10]:
predict(5)

average examples number: 4.6767676767676765
accuracy: 0.7474747474747475


In [11]:
predict(4)

average examples number: 3.787878787878788
accuracy: 0.7676767676767676


In [12]:
predict(3)

average examples number: 2.8686868686868685
accuracy: 0.7474747474747475


## Train of thought

In [13]:
def create_prompt_with_tot(acronym, text, options, examples):
    system = """Tu es un modèle expert en expansion d'acronymes ferroviaires.
Ton rôle est d'identifier la ou les définitions correctes d'un acronyme dans un texte. Analyse synthètiquement le text et les options. 
Après termine en ecrivant les indeces des accronymes corrects sous la forme d'une liste de python. \nExemples:"""

    for idx in examples: 
        example = train.iloc[idx]
        system += f'\nTexte exemple : "{example["text"]}\nAcronyme: {example["acronym"]}"\nOptions: '
        for j, opt in enumerate(example['options'].keys()):
            system += f'\n{j}. : {opt}'
        system += f'\nReponse correcte : {[i for i, value in enumerate(example["options"].values()) if value]}\n'
    user = f'Texte : "{text}"\nAcronyme : {acronym}\n'
    for i, opt in enumerate(options):
        user += f"Option {i} : {opt}\n"
    user += "Analyse chaqu'une des options, termine avec une réponse pour chaque option sous le format indiqué"
    prompt = [
    {"role": "system", "content": system},
    {"role": "user", "content": user}]
    chat = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
    return chat

create_prompt_with_tot(test.iloc[0]['acronym'], test.iloc[0]['text'], test.iloc[0]['options'].keys(), top_k_examples(5)[0])

'<|im_start|>system\nTu es un modèle expert en expansion d\'acronymes ferroviaires.\nTon rôle est d\'identifier la ou les définitions correctes d\'un acronyme dans un texte. Analyse synthètiquement le text et les options. \nAprès termine en ecrivant les indeces des accronymes corrects sous la forme d\'une liste de python. \nExemples:\nTexte exemple : "Bif 2 Lyon / 2 bisZone télécommandée par le PCD de DijonFa\nAcronyme: PCD"\nOptions: \n0. : poste de commande à distance\n1. : Protection de courte durée : méthode de protection des opérations engageant fugitivement le gabarit d\'une voie à l\'occasion des travaux de maintenance de l\'infrastructure\n2. : Potentiel Cadre Dirigeant\nReponse correcte : [0]\n<|im_end|>\n<|im_start|>user\nTexte : "Poste de commande à distance de Saint Sulpice Laurière (PCD) Article A107 Installations de surveillance du service voyageurs Réservé Article A108 Couverture des obstacles "\nAcronyme : PCD\nOption 0 : Protection de courte durée : méthode de protecti

In [18]:
import re
def extract_predicted_ids_tot(outputs):
    predicted_ids = []
    for output in outputs:
        # Get the text after [/INST]
        text = output[0]["generated_text"].split("<|im_start|>assistant")[1] #
        
        ids_for_this_output = []
 
        bracket_contents = re.findall(r'\[(.*?)\]', text)
        
        for content in bracket_contents:
            # Find all numbers within each bracket content
            numbers = re.findall(r'\d+', content)
        
        # Convert to ints safely, remove duplicates, and filter < 15
        ids = list(set(int(float(i)) for i in numbers if float(i) < 15))
        
        predicted_ids.append(ids)
    return predicted_ids

In [19]:
def predict_with_tot(k=4):
    exampless = top_k_examples(k)
    sum = 0
    for examples in exampless:
        sum+= len(examples)
    inputs = []
    for indexx, row in test.iterrows():
        prompt = create_prompt_with_tot(row['acronym'], row['text'], row['options'].keys(), exampless[indexx])
        inputs.append(prompt)
    outputs = pipe(inputs, temperature=0, max_new_tokens=768, do_sample=False, batch_size=4)
    return outputs

In [20]:
outputs = predict_with_tot(4)

In [21]:
outputs[5][0]["generated_text"].split("<|im_start|>assistant")[1]

'\nDans le texte fourni, l\'acronyme "PAR" est défini comme "poste d’aiguillage et de régulation". Cela correspond directement à l\'option 0 qui donne une définition similaire, bien que l\'option 0 ajoute des détails supplémentaires sur ses fonctions.\n\nLes autres options ne correspondent pas à la définition donnée dans le texte :\n\n- Option 1 : "PONT DE L\'ARCHE" n\'est pas lié au contexte ferroviaire du texte.\n- Option 2 : "Plan d\'action régional" ne correspond pas à la définition du texte.\n- Option 3 : "Plan d\'action régularité" ne correspond pas non plus à la définition du texte.\n\nRéponse correcte : [0]'

In [22]:
predicted_ids = extract_predicted_ids_tot(outputs)
correct = 0
for index, ids in enumerate(predicted_ids):
    if ids == test['answers'].iloc[index]:
        correct += 1
accuracy = correct / len(predicted_ids)
print(f"accuracy: {accuracy}")

accuracy: 0.7676767676767676
